In [1]:
from retinaface.interface import RETINAFACE
from arcface.interface import ARCFACE
import logging
import numpy as np
import cv2
import torch
logger = logging.Logger('inference')
import time

retina_trt_engine_path = '/data/Model_Management/engines/retinanet_fp16/retinanet_best.trt' 
    
arc_trt_engine_path = '/data/Model_Management/engines/arcface_fp16/arcface_best.trt' 

video_path = './3_talk.mp4'
output_file = './out.mp4'

retina = RETINAFACE(logger)
retina.load(retina_trt_engine_path)

arc = ARCFACE(logger)
arc.load(arc_trt_engine_path)

print('done')

[TensorRT] WARNING: The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. TensorRT maintains only a single logger pointer at any given time, so the existing value, which can be retrieved with getLogger(), will be used instead. In order to use a new logger, first destroy all existing builder, runner or refitter objects.



done


In [2]:
import os 
fileEx = r'.png'
img_path = './test'
img_list = [file for file in os.listdir(img_path) if file.endswith(fileEx)]
label_vector_dict = {}
for img_file in img_list : 
    
    img_full_path = f'{img_path}/{img_file}'

    img = cv2.imread(img_full_path, cv2.IMREAD_COLOR)
    infe_frame = torch.from_numpy(img).to(torch.device("cuda")).permute(2, 0, 1) 
    input_data = dict()
    input_data["framedata"] = {"frame":infe_frame}
    input_data["bbox"] = [0,0,infe_frame.shape[2],infe_frame.shape[1]]
    input_data["scenario"] = "s"   
    input_data["data"] = None   
    input_data_batch = [input_data]

    output = retina.run_inference(input_data_batch,[])
    arc_output = arc.run_inference(output[0],[]) 


    label_vector_dict[img_file] = [arc_output[0][0]['data']['label']]

vector_dict = {}
vector_dict['label_list'] = [ label[0] for label, _ in label_vector_dict.items()]
vector_dict['label_vectors'] = np.array([ vectors for _, vectors in label_vector_dict.items()])
vector_dict['label_list']

/opt/conda/lib/python3.7/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:4216: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "


['3', '1', '2']

In [6]:
cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
fps = cap.get(cv2.CAP_PROP_FPS)

vid_writer = cv2.VideoWriter(
    output_file, cv2.VideoWriter_fourcc(*'DIVX'), fps, (int(width), int(height))
)

pickle_dict = {}
for l in vector_dict['label_list'] :
    pickle_dict[int(l)] = []
total_time = 0
cnt = 0
result_batch = []
if cap.isOpened() :    
    while True:
        cnt+=1
        ret, frame = cap.read()
        if ret == False :
            break
        if cnt < 0:
            continue
#         elif cnt > 3:
#             break

        #현재 프레임
        num_frame = round(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        print('num_frame',num_frame)
        infe_frame = torch.from_numpy(frame).to(torch.device("cuda")).permute(2, 0, 1) 
        input_data = dict()
        input_data["framedata"] = {"frame":infe_frame}
        input_data["bbox"] = [0,0,infe_frame.shape[2],infe_frame.shape[1]]
        input_data["scenario"] = "s"   
        input_data["data"] = None   
        input_data_batch = [input_data]

        output = retina.run_inference(input_data_batch,[])
        arc_output = arc.run_inference(output[0],[]) 
        for out in arc_output[0] : 
            label_list = vector_dict['label_list']
            label_vectors = vector_dict['label_vectors']

            best_s_label = '81'
            best_score = 0

            targets = np.array(out['data']['label'])
            scores = label_vectors.dot(targets)/(np.linalg.norm(label_vectors, axis=2) * np.linalg.norm(targets))
            
            result = np.mean(scores,axis=1)

            b_score_idx = np.argmax(result)
            best_s_label = label_list[b_score_idx]
            best_score = result[b_score_idx]
            if best_score < 0.3 : 
                continue
            print(best_s_label,best_score)
            box1,box2,box3,box4 = out['bbox']
            draw_bboxes = int(box1), int(box2), int(box3), int(box4)
            tl = (int(draw_bboxes[0]), int(draw_bboxes[1]))
            br = (int(draw_bboxes[2]), int(draw_bboxes[3]))
            ty1 = min(height, int(draw_bboxes[1]))
         
            label = int(best_s_label)
            if label == 1 : 
                color = [0,255,255]
            if label == 2 : 
                color = [255,0,255]
            if label == 3 : 
                color = [255,0,0]
            cv2.rectangle(frame, tl, br, color, 2)
            
            pickle_dict[label].append(targets)


        vid_writer.write(frame)
        ch = cv2.waitKey(1)
        if ch == 27 or ch == ord("q") or ch == ord("Q"):
            break
vid_writer.release()

            








OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


num_frame 0
1 0.6433497
3 0.57227325
2 0.3403455
num_frame 1
1 0.6310455
2 0.33858338
3 0.5809872
num_frame 2
3 0.57573473
2 0.32306564
1 0.6475418
num_frame 3
2 0.42071232
3 0.6421604
1 0.6405339
num_frame 4
2 0.46397167
1 0.6440687
3 0.62082326
num_frame 5
2 0.46556014
1 0.6446257
3 0.62140566
num_frame 6
2 0.5899053
1 0.66914785
3 0.5585224
num_frame 7
2 0.5832401
3 0.5527519
1 0.6634378
num_frame 8
2 0.6154222
3 0.53878206
1 0.6444705
num_frame 9
2 0.6282645
3 0.519366
1 0.62537473
num_frame 10
2 0.62876785
3 0.5238729
1 0.6243632
num_frame 11
2 0.5764958
1 0.6011936
3 0.5102681
num_frame 12
2 0.46936804
3 0.5357945
1 0.5935465
num_frame 13
3 0.54508626
2 0.48188674
1 0.6300334
num_frame 14
3 0.57148355
1 0.65079117
2 0.49008796
num_frame 15
3 0.5725479
1 0.64957786
2 0.48758677
num_frame 16
3 0.5996793
1 0.6581365
2 0.5547498
num_frame 17
2 0.6169666
3 0.63746256
1 0.66742873
num_frame 18
2 0.6080302
3 0.6241145
1 0.67686236
num_frame 19
2 0.6229247
3 0.6074447
1 0.66948867
num_fr

num_frame 166
3 0.5114634
2 0.7130985
1 0.6984848
num_frame 167
3 0.47304764
2 0.70788413
1 0.680317
num_frame 168
2 0.69026387
3 0.49117768
1 0.63798845
num_frame 169
2 0.6684328
3 0.52887213
1 0.6490408
num_frame 170
2 0.6677969
3 0.5265304
1 0.64882016
num_frame 171
2 0.6252337
3 0.5250939
1 0.6747232
num_frame 172
3 0.4992931
2 0.6151657
1 0.65434253
num_frame 173
3 0.5134381
2 0.6440565
1 0.6441136
num_frame 174
3 0.5089781
1 0.6632704
2 0.6879845
num_frame 175
3 0.50577384
1 0.6665874
2 0.6804632
num_frame 176
3 0.46096936
2 0.65635574
1 0.68474567
num_frame 177
3 0.48426968
2 0.6410261
1 0.6636773
num_frame 178
2 0.67022216
3 0.49020946
1 0.62664557
num_frame 179
2 0.67596734
3 0.4933249
1 0.65012425
num_frame 180
2 0.67107934
3 0.4920506
1 0.6550877
num_frame 181
3 0.5054467
2 0.66723126
1 0.66364235
num_frame 182
2 0.6832671
3 0.5369653
1 0.6811692
num_frame 183
2 0.69057596
3 0.53204495
1 0.68314797
num_frame 184
3 0.5498928
2 0.6755878
1 0.6822878
num_frame 185
3 0.5578458
2

3 0.53607583
2 0.70071495
1 0.69876486
num_frame 330
3 0.53479266
2 0.70046884
1 0.6955452
num_frame 331
2 0.6647586
3 0.5308657
1 0.67772216
num_frame 332
2 0.68111
3 0.54253334
1 0.68192756
num_frame 333
2 0.68169284
3 0.57303405
1 0.64768875
num_frame 334
2 0.66822755
3 0.5503497
1 0.6558853
num_frame 335
2 0.6747881
3 0.55172044
1 0.65481925
num_frame 336
2 0.6943894
3 0.5400171
1 0.66954947
num_frame 337
2 0.71451277
3 0.54656863
1 0.6929031
num_frame 338
2 0.68450326
3 0.57367
1 0.6970647
num_frame 339
2 0.59340113
3 0.55918163
1 0.7079342
num_frame 340
2 0.5905033
3 0.566647
1 0.70474243
num_frame 341
2 0.63617593
1 0.7318466
3 0.5917457
num_frame 342
2 0.7334828
1 0.73132634
3 0.5627184
num_frame 343
1 0.7295824
2 0.628708
3 0.5486782
num_frame 344
2 0.67385894
1 0.73906803
3 0.54642475
num_frame 345
2 0.68171865
1 0.745263
3 0.53943163
num_frame 346
2 0.7017556
1 0.7290722
3 0.5687144
num_frame 347
2 0.6880101
3 0.5746357
1 0.7236952
num_frame 348
2 0.72940916
3 0.59731704
1 0

3 0.5528413
1 0.6914591
2 0.69378847
num_frame 493
1 0.6972958
3 0.5991095
2 0.6998798
num_frame 494
1 0.70956224
3 0.6214541
2 0.68721265
num_frame 495
1 0.7079968
3 0.61398935
2 0.7030847
num_frame 496
1 0.7084097
3 0.6267968
2 0.6809402
num_frame 497
1 0.7059629
3 0.6477015
2 0.67684877
num_frame 498
1 0.70663744
3 0.643199
2 0.63893926
num_frame 499
1 0.7219502
3 0.5969667
2 0.6881004
num_frame 500
1 0.7215801
3 0.6042657
2 0.68556005
num_frame 501
1 0.69257486
2 0.6783014
3 0.58210033
num_frame 502
1 0.6963902
3 0.5887333
2 0.6788304
num_frame 503
1 0.69444084
3 0.632906
2 0.67889273
num_frame 504
1 0.6876956
3 0.6157859
2 0.67551017
num_frame 505
1 0.69169146
3 0.61799055
2 0.67586535
num_frame 506
1 0.6481732
3 0.6451804
2 0.7070183
num_frame 507
1 0.65281385
3 0.6535161
2 0.75112987
num_frame 508
1 0.6746391
3 0.6563382
2 0.7276046
num_frame 509
1 0.6317914
2 0.7218064
3 0.64668447
num_frame 510
1 0.63222337
2 0.73532104
3 0.6359819
num_frame 511
1 0.60971296
2 0.7180405
3 0.63

...

num_frame 1364
2 0.80085695
3 0.4325515
num_frame 1365
2 0.8063592
3 0.42865303
num_frame 1366
2 0.79694825
3 0.42007247
num_frame 1367
2 0.8085769
3 0.3920127
num_frame 1368
2 0.8049024
3 0.40616602
num_frame 1369
2 0.80960155
3 0.41148353
num_frame 1370
2 0.8099499
3 0.42135894
num_frame 1371
2 0.82181984
3 0.41238657
num_frame 1372
2 0.8026635
3 0.43800786
num_frame 1373
2 0.8086046
3 0.416745
num_frame 1374
2 0.8106746
3 0.40856594
num_frame 1375
2 0.815004
3 0.38834155
num_frame 1376
2 0.80571806
3 0.40413982
num_frame 1377
2 0.8023517
3 0.4042554
num_frame 1378
2 0.7872476
3 0.39669922
num_frame 1379
2 0.7826093
3 0.39896864
num_frame 1380
2 0.7809229
3 0.39950058
num_frame 1381
2 0.7734719
3 0.40301612
num_frame 1382
2 0.768526
3 0.3921977
num_frame 1383
2 0.767373
3 0.399824
num_frame 1384
2 0.77554405
3 0.39709818
num_frame 1385
2 0.7740334
3 0.39904657
num_frame 1386
2 0.7797309
3 0.40652418
num_frame 1387
2 0.77590036
3 0.38914895
num_frame 1388
2 0.7959418
3 0.4112374
num_f

3 0.79271626
num_frame 1649
3 0.7671947
num_frame 1650
3 0.7607008
num_frame 1651
3 0.73389494
num_frame 1652
3 0.71530795
num_frame 1653
1 0.95770866
num_frame 1654
1 0.95787096
num_frame 1655
1 0.9595565
num_frame 1656
1 0.9611907
num_frame 1657
1 0.95951897
num_frame 1658
1 0.9617423
num_frame 1659
1 0.96285635
num_frame 1660
1 0.96206623
num_frame 1661
1 0.96665525
num_frame 1662
1 0.9850483
num_frame 1663
1 0.9692907
num_frame 1664
1 0.96731985
num_frame 1665
1 0.966706
num_frame 1666
1 0.96095634
num_frame 1667
1 0.962259
num_frame 1668
1 0.9518942
num_frame 1669
1 0.9364655
num_frame 1670
1 0.9347736
num_frame 1671
1 0.9264984
num_frame 1672
1 0.91942495
num_frame 1673
1 0.8612844
num_frame 1674
1 0.8816109
num_frame 1675
1 0.88607347
num_frame 1676
1 0.9057594
num_frame 1677
1 0.91081345
num_frame 1678
1 0.93262106
num_frame 1679
1 0.935684
num_frame 1680
1 0.9374463
num_frame 1681
1 0.9399202
num_frame 1682
1 0.9433878
num_frame 1683
1 0.9453722
num_frame 1684
1 0.94436425
num

1 0.6132894
3 0.41348562
num_frame 1915
1 0.624017
3 0.43247974
num_frame 1916
1 0.58775043
3 0.43526235
num_frame 1917
1 0.57706547
3 0.43853635
num_frame 1918
1 0.6301695
3 0.46571624
num_frame 1919
1 0.6781247
3 0.46734473
num_frame 1920
1 0.67671454
3 0.44940037
num_frame 1921
1 0.6659997
3 0.4689423
num_frame 1922
1 0.62479854
3 0.4479134
num_frame 1923
1 0.60548323
3 0.46435228
num_frame 1924
1 0.5904088
3 0.45574027
num_frame 1925
1 0.5893972
3 0.44830552
num_frame 1926
1 0.63113296
3 0.42307153
num_frame 1927
1 0.6616989
3 0.43659845
num_frame 1928
1 0.6831581
3 0.47571343
num_frame 1929
1 0.69317716
3 0.4592852
num_frame 1930
1 0.6878092
3 0.4547747
num_frame 1931
1 0.689117
3 0.45708716
num_frame 1932
1 0.65217316
3 0.45615247
num_frame 1933
1 0.620385
3 0.45739162
num_frame 1934
1 0.64305645
3 0.4712407
num_frame 1935
1 0.64874893
3 0.459228
num_frame 1936
1 0.67005754
3 0.46732923
num_frame 1937
1 0.6949998
3 0.4668711
num_frame 1938
1 0.72366524
3 0.4286116
num_frame 1939


2 0.7797334
num_frame 2183
2 0.7818525
num_frame 2184
2 0.76515955
num_frame 2185
2 0.7695242
num_frame 2186
2 0.7749852
num_frame 2187
2 0.7771632
num_frame 2188
2 0.7703232
num_frame 2189
2 0.7836865
num_frame 2190
2 0.7749079
num_frame 2191
2 0.7666279
num_frame 2192
2 0.760208
num_frame 2193
2 0.7672188
num_frame 2194
2 0.76972574
num_frame 2195
2 0.76592237
num_frame 2196
2 0.7518735
num_frame 2197
2 0.7613862
num_frame 2198
2 0.77109885
num_frame 2199
2 0.78373533
num_frame 2200
2 0.7847268
num_frame 2201
2 0.7902081
num_frame 2202
2 0.798065
num_frame 2203
2 0.77951866
num_frame 2204
2 0.7861142
num_frame 2205
2 0.7844314
num_frame 2206
2 0.7996397
num_frame 2207
2 0.7831472
num_frame 2208
2 0.78311217
num_frame 2209
2 0.772688
num_frame 2210
2 0.7783169
num_frame 2211
2 0.7803786
num_frame 2212
2 0.7784806
num_frame 2213
2 0.7828372
num_frame 2214
1 0.763853
num_frame 2215
1 0.76715624
num_frame 2216
1 0.7538051
num_frame 2217
1 0.7188838
num_frame 2218
1 0.5592687
num_frame 22

2 0.6018961
3 0.33069098
num_frame 2489
2 0.6133826
3 0.3204386
num_frame 2490
2 0.6193429
3 0.32196683
num_frame 2491
2 0.60237485
3 0.31805074
num_frame 2492
2 0.6138237
3 0.3326586
1 0.30821776
num_frame 2493
2 0.61614126
3 0.33479086
1 0.3819042
num_frame 2494
2 0.629541
1 0.42602643
3 0.32593346
num_frame 2495
2 0.6227126
1 0.4198827
3 0.32638815
num_frame 2496
2 0.6382429
1 0.4543871
3 0.3236597
num_frame 2497
2 0.63701355
1 0.43759954
3 0.33030832
num_frame 2498
2 0.6312791
1 0.40942377
3 0.3082084
num_frame 2499
2 0.63832384
1 0.41096723
3 0.36086252
num_frame 2500
2 0.6401369
1 0.41301942
3 0.36336604
num_frame 2501
2 0.64106524
1 0.39811048
3 0.34869704
num_frame 2502
2 0.64955544
1 0.3812107
3 0.39209056
num_frame 2503
2 0.6484188
1 0.3626711
3 0.32107863
num_frame 2504
2 0.64818525
1 0.37560695
3 0.3504564
num_frame 2505
2 0.6526671
1 0.36141998
3 0.36084858
num_frame 2506
2 0.6543823
1 0.34997344
3 0.41754326
num_frame 2507
2 0.654997
1 0.4130077
3 0.39351165
num_frame 250

In [7]:
import pickle
for label_id, vec_list in pickle_dict.items() :
    save_path = f'./test/{label_id}_0.3.pickle'
    data = {
        str(label_id) : vec_list
            }
    with open(save_path, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [5]:
data_path = f'./test/1.pickle'
with open(data_path, 'rb') as f:
    data = pickle.load(f)
    
for k, vs in data.items():
    print(len(vs))
    for v in vs:
        print(v.shape)

2507
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(